# 深度学习基本流程

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_basic_process_deep_learning.ipynb)&emsp;[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_basic_process_deep_learning.py)&emsp;[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/beginner/basic_process_deep_learning.ipynb)

本章节贯穿MindSpore深度学习的基本流程，实现深度学习中的常见任务。

## 配置运行信息

通过导入MindSpore的`context`模块，使用`context.set_context`函数来配置运行需要的硬件类型、运行模式、后端等信息，代码样例如下：

In [12]:
from mindspore import context

context.set_context(mode=context.GRAPH_MODE)

在样例中，我们通过配置使用图模式运行，详细参数说明请参见文档[context.set_context](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.context.html)。

## 下载并处理数据集

数据集对于模型训练非常重要，好的数据集可以有效提高训练精度和效率。示例中用到的MNIST数据集是由10类28∗28的灰度图片组成，训练数据集包含60000张图片，测试数据集包含10000张图片。你可以从[MNIST数据集下载页面](http://yann.lecun.com/exdb/mnist/)下载，解压后按下方目录结构放置。

MindVision套件提供了用于下载并处理MNIST数据集的`Mnist`模块，以下示例代码将数据集下载、解压到指定位置并进行数据处理：

In [13]:
from mindvision.classification.dataset import Mnist

# 下载并处理MNIST数据集
download_train = Mnist(path="./mnist", split="train", batch_size=32, repeat_num=1, shuffle=True, resize=32, download=True)
download_eval = Mnist(path="./mnist", split="test", batch_size=32, resize=32, download=True)

dataset_train = download_train.run()
dataset_eval = download_eval.run()

参数说明：

- path：数据集路径。
- split：数据集类型，支持`train`、 `test`、`infer`，默认为`train`。
- batch_size：每个训练批次设定的数据大小，默认为`32`。
- repeat_num：训练时遍历数据集的次数，默认为`1`。
- shuffle：是否需要将数据集随机打乱（可选参数）。
- resize：输出图像的图像大小，默认为`32*32`。
- download：是否需要下载数据集，默认为`False`。

下载的数据集文件的目录结构如下：

```text
./mnist/
├── test
│   ├── t10k-images-idx3-ubyte
│   └── t10k-labels-idx1-ubyte
└── train
    ├── train-images-idx3-ubyte
    └── train-labels-idx1-ubyte
```

## 创建网络模型

MindVision套件提供了LeNet网络模型接口`lenet`， 定义网络模型如下：

In [14]:
from mindvision.classification.models import lenet

network = lenet(num_classes=10, pretrained=False)

## 初始化模型参数

要训练神经网络模型，需要定义损失函数和优化器函数。

- MindSpore支持的损失函数有`SoftmaxCrossEntropyWithLogits`、`L1Loss`、`MSELoss`等，这里使用交叉熵损失函数`SoftmaxCrossEntropyWithLogits`。
- MindSpore支持的优化器函数有`Adam`、`AdamWeightDecay`、`Momentum`等，这里使用优化器函数`Momentum`。

In [15]:
import mindspore.nn as nn
from mindspore.train import Model

# 定义损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
# 定义优化器函数
net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)

# 初始化模型参数
model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={'acc'})

## 训练及保存模型

MindSpore提供了回调Callback机制，可以在训练过程中执行自定义逻辑，这里以框架提供的`ModelCheckpoint`为例。
`ModelCheckpoint`可以保存网络模型和参数，以便进行后续的Fine-tuning（微调）操作。

In [16]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

# 设置模型保存参数
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
# 应用模型保存参数
ckpoint = ModelCheckpoint(prefix="lenet", directory="./lenet", config=config_ck)

通过MindSpore提供的`model.train`接口可以方便地进行网络的训练，`LossMonitor`可以监控训练过程中`loss`值的变化。`dataset_sink_mode`用于控制数据是否下沉，数据下沉是指数据通过通道直接传送到Device上，可以加快训练速度，`dataset_sink_mode`为`True`表示数据下沉，否则为非下沉。

In [ ]:
from mindvision.engine.callback import LossMonitor

model.train(1, dataset_train, callbacks=[ckpoint, LossMonitor(125)], dataset_sink_mode=False)

训练过程中会打印loss值，类似下图。loss值会波动，但总体来说loss值会逐步减小，精度逐步提高。每个人运行的loss值有一定随机性，不一定完全相同。
训练过程中loss打印示例如下：

```text
Epoch:[  0/  1],   step:[ 1856/ 1875],  loss:[0.131/1.448],  time:7.942,  lr:125.00000.
Epoch:[  0/  1],   step:[ 1857/ 1875],  loss:[0.018/1.447],  time:8.058,  lr:125.00000.
...
Epoch:[  0/  1],   step:[ 1858/ 1875],  loss:[0.025/1.446],  time:8.029,  lr:125.00000.
Epoch:[  0/  1],   step:[ 1859/ 1875],  loss:[0.023/1.445],  time:7.952,  lr:125.00000.
Epoch:[  0/  1],   step:[ 1860/ 1875],  loss:[0.133/1.445],  time:8.030,  lr:125.00000.

```

通过模型运行测试数据集得到的结果，验证模型的泛化能力：

1. 使用`model.eval`接口读入测试数据集。
2. 使用保存后的模型参数进行推理。

In [18]:
acc = model.eval(dataset_eval, dataset_sink_mode=False)
print("{}".format(acc))

{'acc': 0.9733573717948718}


可以在打印信息中看出模型精度数据，示例中精度数据达到95.5%，模型质量良好。随着网络迭代次数增加，模型精度会进一步提高。

## 加载模型

In [ ]:
from mindspore import load_checkpoint, load_param_into_net
# 加载已经保存的用于测试的模型
param_dict = load_checkpoint("./lenet/lenet-1_1875.ckpt")
# 加载参数到网络中
load_param_into_net(network, param_dict)

> 阅读更多有关[mindspore加载模型](https://www.mindspore.cn/tutorials/zh-CN/master/save_load_model.html#id3)的信息。

## 验证模型

我们使用生成的模型进行单个图片数据的分类预测，具体步骤如下：

> 被预测的图片会随机生成，每次运行结果可能会不一样。

In [20]:
import numpy as np
from mindspore import Tensor

mnist = Mnist("./mnist", split="train", batch_size=1, resize=32)
dataset_infer = mnist.run()
ds_test = dataset_infer.create_dict_iterator()
data = next(ds_test)
image = data["image"].asnumpy()
label = data["label"].asnumpy()

# 使用函数model.predict预测image对应分类
output = model.predict(Tensor(data['image']))
predicted = np.argmax(output.asnumpy(), axis=1)

# 输出预测分类与实际分类
print(f'Predicted: "{predicted[0]}", Actual: "{label[0]}"')

Predicted: "1", Actual: "1"
